# Step 1: Import Necessary Libraries

In [1]:
import geopandas as gpd
import json

# Step 2: Load Cleaned Data

In [3]:
# Load cleaned GeoJSON data
state_boundaries = gpd.read_file("state_geojson.geojson")
lga_boundaries = gpd.read_file("lga_geojson.geojson")
ward_boundaries = gpd.read_file("ward_geojson.geojson")

# Step 3: Create State-to-LGA and LGA-to-Ward Relationships

In [6]:
# Create mappings
state_to_lga = lga_boundaries.groupby("statename")["lganame"].apply(list).to_dict()
lga_to_ward = ward_boundaries.groupby("lganame")["wardname"].apply(list).to_dict()

# Generate HTML with full state names
state_options = "".join([f'<option value="{state}">{state}</option>' for state in state_to_lga.keys()])

# Step 4: Convert GeoDataFrames to GeoJSON

In [7]:
# Convert to GeoJSON format
state_geojson = state_boundaries.to_crs(epsg=4326).to_json()
lga_geojson = lga_boundaries.to_crs(epsg=4326).to_json()
ward_geojson = ward_boundaries.to_crs(epsg=4326).to_json()


# Step 5: Generate Ward layer

In [24]:
# Read the GeoJSON file
ward_gdf = gpd.read_file('ward_geojson_corrected.geojson')

# Display the column names (fields)
print("Available fields:")
print(ward_gdf.columns.tolist())

# Display the first few rows to see example values
print("\nSample data:")
print(ward_gdf.head())

# If you want to see all unique values for a specific field
print("\nUnique values for a field (e.g., if there's a 'type' field):")
if 'type' in ward_gdf.columns:
    print(ward_gdf['type'].unique())

# Get summary statistics for numeric fields
print("\nSummary statistics for numeric fields:")
print(ward_gdf.describe())

# Check the data types of each field
print("\nData types:")
print(ward_gdf.dtypes)

# Save fields to a JSON file for reference
field_info = {
    'fields': ward_gdf.columns.tolist(),
    'dtypes': {col: str(dtype) for col, dtype in zip(ward_gdf.columns, ward_gdf.dtypes)}
}
with open('ward_fields.json', 'w') as f:
    json.dump(field_info, f, indent=4)
print("\nField information saved to ward_fields.json")

Available fields:
['wardname', 'wardcode', 'lganame', 'lgacode', 'statename', 'statecode', 'geometry']

Sample data:
     wardname  wardcode        lganame lgacode    statename statecode  \
0       Ikuru  RVSNDN06         Andoni   33005       Rivers        RI   
1  Aktward 17  RVSABM17     Akuku Toru   33004       Rivers        RI   
2   Aktward 1  RVSABM01     Akuku Toru   33004       Rivers        RI   
3     Ward 12  CRSANA12  Calabar South    9009  Cross River        CR   
4     Koluama  BYSSPR09  Southern Ijaw    6007      Bayelsa        BY   

                                            geometry  
0  POLYGON ((7.39065 4.43648, 7.38503 4.47910, 7....  
1  POLYGON ((6.71866 4.61093, 6.71477 4.62050, 6....  
2  POLYGON ((6.82083 4.63792, 6.83814 4.61875, 6....  
3  POLYGON ((8.30619 4.98152, 8.30968 4.97872, 8....  
4  POLYGON ((5.98169 4.42706, 5.97855 4.41279, 5....  

Unique values for a field (e.g., if there's a 'type' field):

Summary statistics for numeric fields:
          wa

# Senatorial districts

In [8]:
import pandas as pd

# Load the Senatorial Constituencies spreadsheet
file_path = r"C:\Users\femio\Downloads\Name-of-Senatorial-DistrictsFederal-and-State-Constituencies-Nationwide-1.xlsx"

# Load all sheets to check the structure
xls = pd.ExcelFile(file_path)
xls.sheet_names

# Load the Senatorial Constituencies sheet
senatorial_constituencies = pd.read_excel(file_path, sheet_name="SEN. DIST.")

# Display the first few rows to see example values
senatorial_constituencies.head()



,S/N,NAME OF SENATORIAL DISTRICT,CODE,COMPOSITION,STATE,NAME OF COLLATION CENTRE
0,1,ABIA NORTH,SD/001/AB,"UMUNNEOCHI, ISUKWUATO, OHAFIA, AROCHUKWU, BENDE",ABIA,COUNCIL HALL OHAFIA LGA HQS
1,2,ABIA CENTRAL,SD/002/AB,"UMUAHIA NORTH, UMUAHIA SOUTH, IKWUANO, ISIALA ...",ABIA,COUNCIL HALL UMUAHIA NORTH LGA HQS
2,3,ABIA SOUTH,SD/003/AB,"ABA NORTH, ABA SOUTH, UGWUNAGBO, OBINGWA, UKW...",ABIA,ABA TOWN HALL ABA SOUTH LGA
3,S/N,NAME OF SENATORIAL DISTRICT,CODE,COMPOSITION,AKWA IBOM,NAME OF COLLATION CENTRE
4,7,AKWA IBOM NORTH EAST,SD/007/AK,"ETINAN, IBESIKPO ASUTAN, IBIONO IBOM, ITU, NSI...",AKWA IBOM,INEC OFFICE UYO LGA


In [ ]:
# Remove duplicate header row (detected at index 3)
df_cleaned = senatorial_constituencies[senatorial_constituencies["S/N"] != "S/N"].reset_index(drop=True)

# Rename columns for clarity
df_cleaned = df_cleaned.rename(columns={
    "S/N": "Serial_Number",
    "NAME OF SENATORIAL DISTRICT": "Senatorial_District",
    "CODE": "District_Code",
    "COMPOSITION": "LGAs",
    "STATE": "State",
    "NAME OF COLLATION CENTRE": "Collation_Centre"
})

# Trim whitespace and standardize text formatting
df_cleaned = df_cleaned.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Convert LGAs from comma-separated text to a list
df_cleaned["LGAs"] = df_cleaned["LGAs"].apply(lambda x: [lga.strip() for lga in x.split(",")] if isinstance(x, str) else [])

# Display cleaned data
from IPython.display import display
display(df_cleaned)


C:\Users\femio\AppData\Local\Temp\ipykernel_28612\2075056213.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df_cleaned.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Serial_Number,Senatorial_District,CODE,LGAs,State,Collation_Centre
0,1.0,ABIA NORTH,SD/001/AB,"[UMUNNEOCHI, ISUKWUATO, OHAFIA, AROCHUKWU, BENDE]",ABIA,COUNCIL HALL OHAFIA LGA HQS
1,2.0,ABIA CENTRAL,SD/002/AB,"[UMUAHIA NORTH, UMUAHIA SOUTH, IKWUANO, ISIALA...",ABIA,COUNCIL HALL UMUAHIA NORTH LGA HQS
2,3.0,ABIA SOUTH,SD/003/AB,"[ABA NORTH, ABA SOUTH, UGWUNAGBO, OBINGWA, UKW...",ABIA,ABA TOWN HALL ABA SOUTH LGA
3,7.0,AKWA IBOM NORTH EAST,SD/007/AK,"[ETINAN, IBESIKPO ASUTAN, IBIONO IBOM, ITU, NS...",AKWA IBOM,INEC OFFICE UYO LGA
4,8.0,AKWA IBOM NORTH WEST,SD/008/AK,"[ABAK, ESSIEN UDIM, ETIM EKPO, IKA, IKONO, IKO...",AKWA IBOM,INEC OFFICE IKOT EKPENELGA
...,...,...,...,...,...,...
148,NaN,NaN,NaN,[],NaN,NaN
149,NaN,NaN,NaN,[],NaN,NaN
150,NaN,NaN,NaN,[],NaN,NaN
151,NaN,NaN,NaN,[],NaN,NaN


In [18]:
# Check for null values in df_cleaned
null_values = df_cleaned.isnull().sum()
print(null_values) 
# Remove null values 
df_cleaned = df_cleaned.dropna()
len(df_cleaned) # Check the number of rows after removing null values

Serial_Number          0
Senatorial_District    0
   CODE                0
LGAs                   0
State                  0
Collation_Centre       0
dtype: int64


109

In [27]:
print(type(df_cleaned.loc[0, "LGAs"]))

df_cleaned.head()

# Delineat LGAs into separate rows
df_lga = df_cleaned.explode("LGAs").reset_index(drop=True)

# Display the first few rows to see the result
df_lga.head()

# export the cleaned data to a new Excel file
df_lga.to_excel("senatorial_constituencies_cleaned.xlsx", index=False)

<class 'list'>


# Attach Ward Geometries to LGAs

In [26]:
# join the cleaned data with the ward boundaries.
# Merge the dataframes based on the 'LGAs' column
df_merged = pd.merge(df_lga, ward_gdf, left_on='LGAs', right_on='lganame')

# Display the first few rows of the merged dataframe
df_merged.head()

,Serial_Number,Senatorial_District,CODE,LGAs,State,Collation_Centre,wardname,wardcode,lganame,lgacode,statename,statecode,geometry


In [1]:
import pandas as pd

df = pd.read_excel('./data/senatorial_constituencies_cleaned.xlsx')
df.to_json('senatorial.json', orient='records')


# checking for inconsistent LGA naming

In [2]:
# Install the ace_tools package
%pip install ace_tools

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import json
import re
from collections import defaultdict

# Load files
senatorial_df = pd.read_excel('data/senatorial_constituencies_cleaned.xlsx')
with open('data/lga_geojson.geojson') as f:
    lga_geojson = json.load(f)

# Function equivalent to normalizeName in mappings.js
def normalize_name(name):
    if not isinstance(name, str):
        return ''  # Return an empty string if the input is not a string
    return re.sub(r'[^a-z0-9 ]', '', re.sub(r'\s+', ' ', name.strip().lower().replace('-', ' ').replace('_', ' ')))

# Build dictionaries for quick lookup
geojson_lgas = {}
for feature in lga_geojson['features']:
    lga_name = feature['properties']['lganame']
    state_name = feature['properties']['statename']
    geojson_lgas[normalize_name(lga_name)] = {
        'original_name': lga_name,
        'state': state_name
    }

# Find unmatched LGAs
unmatched_by_state = defaultdict(list)

for idx, row in senatorial_df.iterrows():
    district = row.get('Senatorial_District') or row.get('district') or row.get('senatorial_district')
    raw_lga = row.get('LGAs') or row.get('lga') or row.get('LGA')
    
    if not raw_lga:
        continue
    
    normalized_lga = normalize_name(raw_lga)
    
    if normalized_lga not in geojson_lgas:
        unmatched_by_state['UNKNOWN'].append({
            'district': district,
            'senatorial_lga': raw_lga
        })
    else:
        state = geojson_lgas[normalized_lga]['state']
        unmatched_by_state[state].append({
            'district': district,
            'senatorial_lga': raw_lga,
            'geojson_lga': geojson_lgas[normalized_lga]['original_name']
        })

# Filter those where names don't exactly match
mismatches = defaultdict(list)
for state, matches in unmatched_by_state.items():
    for match in matches:
        # If state is unknown or names differ (manual inspection case)
        if state == 'UNKNOWN' or match['senatorial_lga'].lower() != match.get('geojson_lga', '').lower():
            mismatches[state].append(match)

# Convert to DataFrame for easy review
df_mismatches = pd.DataFrame([
    {
        'State': state,
        'District': item['district'],
        'Senatorial LGA': item['senatorial_lga'],
        'GeoJSON LGA': item.get('geojson_lga', 'NO MATCH')
    }
    for state, items in mismatches.items() for item in items
])

df_mismatches.head()

df_mismatches.to_excel('unmatched_lgas.xlsx', index=False)


In [26]:
import pandas as pd
import json
from collections import defaultdict

# Load original senatorial mapping
with open("data/senatorial.json", "r") as f:
    senatorial_data = json.load(f)

# Convert to dictionary for easier update
district_to_lgas = defaultdict(list)
for record in senatorial_data:
    district = record['Senatorial_District']
    lga = record['LGAs']
    district_to_lgas[district].append(lga)

# Load unmatched_lgas.xlsx (you just updated this!)
df_unmatched = pd.read_excel("unmatched_lgas.xlsx")

# Loop through and add new LGAs to the correct districts
for _, row in df_unmatched.iterrows():
    district = row['District']
    lga = row['GeoJSON LGA'] 

    if district and lga and lga.lower() != 'nan' and lga not in district_to_lgas[district]:
        district_to_lgas[district].append(lga)

# Rebuild updated list
updated_records = []
for district in sorted(district_to_lgas.keys()):
    # Filter out None values before sorting LGAs
    valid_lgas = [lga for lga in district_to_lgas[district] if lga is not None]
    for lga in sorted(valid_lgas):
        updated_records.append({
            "Senatorial_District": district,
            "LGAs": lga
        })

# Save new senatorial.json
with open("data/senatorial.json", "w") as f:
    json.dump(updated_records, f, indent=2)

print("✅ Updated senatorial.json written!")


✅ Updated senatorial.json written!
